In [1]:
import pandas as pd
cab_df = pd.read_csv("cab_rides.csv",delimiter='\t',encoding = "utf-16")
weather_df = pd.read_csv("weather.csv",delimiter='\t',encoding = "utf-16")
cab_df['date_time'] = pd.to_datetime(cab_df['time_stamp']/1000, unit='s')
weather_df['date_time'] = pd.to_datetime(weather_df['time_stamp'], unit='s')
#merge the datasets to refelect same time for a location
cab_df['merge_date'] = cab_df.source.astype(str) +" - "+ cab_df.date_time.dt.date.astype("str") +" - "+ cab_df.date_time.dt.hour.astype("str")
weather_df['merge_date'] = weather_df.location.astype(str) +" - "+ weather_df.date_time.dt.date.astype("str") +" - "+ weather_df.date_time.dt.hour.astype("str")
weather_df.index = weather_df['merge_date']
merged_df = cab_df.join(weather_df,on=['merge_date'],rsuffix ='_w')
merged_df['rain'].fillna(0,inplace=True)
merged_df = merged_df[pd.notnull(merged_df['date_time_w'])]
merged_df['day'] = merged_df.date_time.dt.dayofweek
merged_df['hour'] = merged_df.date_time.dt.hour


In [2]:
merged_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,...,clouds,pressure,rain,time_stamp_w,humidity,wind,date_time_w,merge_date_w,day,hour
0,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,...,0.39,1022.44,0.0,1.544950e+09,0.74,8.14,2018-12-16 08:45:01,Haymarket Square - 2018-12-16 - 8,6,8
1,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,...,0.39,1022.44,0.0,1.544950e+09,0.74,8.14,2018-12-16 08:45:01,Haymarket Square - 2018-12-16 - 8,6,8
2,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,...,0.39,1022.44,0.0,1.544950e+09,0.74,8.14,2018-12-16 08:45:01,Haymarket Square - 2018-12-16 - 8,6,8
3,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,...,0.39,1022.44,0.0,1.544950e+09,0.74,8.14,2018-12-16 08:45:01,Haymarket Square - 2018-12-16 - 8,6,8
4,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,...,0.39,1022.44,0.0,1.544950e+09,0.74,8.14,2018-12-16 08:45:01,Haymarket Square - 2018-12-16 - 8,6,8


In [6]:
X = merged_df[merged_df.product_id=='lyft_line'][['day','distance','hour','temp','clouds', 'pressure','humidity', 'wind', 'rain']]
y = merged_df[merged_df.product_id=='lyft_line']['price']


#不知道啥意思
X.reset_index(inplace=True)
X = X.drop(columns=['index'])



features = pd.get_dummies(X)


# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(y)

# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [29]:
labels

array([5., 3., 3., ..., 7., 7., 7.])

In [7]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [36]:
import tensorflow as tf


model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, input_shape=(9,), batch_size=32),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(18, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'mse'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(np.transpose(x_test), np.transpose(y_test))


Train on 68280 samples
Epoch 1/5
68280/68280 [==============================] - 6s 89us/sample - loss: 11.1430 - mae: 2.0882 - mse: 11.1429
Epoch 2/5
68280/68280 [==============================] - 5s 70us/sample - loss: 4.3151 - mae: 1.6094 - mse: 4.3151
Epoch 3/5
68280/68280 [==============================] - 5s 69us/sample - loss: 3.4710 - mae: 1.4250 - mse: 3.4710
Epoch 4/5
68280/68280 [==============================] - 5s 70us/sample - loss: 2.9374 - mae: 1.2984 - mse: 2.9374
Epoch 5/5
68280/68280 [==============================] - 5s 70us/sample - loss: 2.5614 - mae: 1.2042 - mse: 2.5614
22761/1 [=======================================================================================================================================================================================================================================================================================================================================================================================================

[1.8537848923465163, 0.98827404, 1.8537843]

In [35]:
x_test.shape

(9, 22761)